# Model PoC

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

In [3]:
logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

In [4]:
log = logging.getLogger(__name__)

In [5]:
from omegaconf import OmegaConf

In [6]:
c = OmegaConf.load("../config/main.yaml")

In [7]:
c.settings.debug = False
c.wandb.enabled = False
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

In [8]:
log.info(OmegaConf.to_yaml(c))

2022-01-29 20:54:52,059 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
  group: default
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
params:
  seed: 440
  n_class: 1
  n_fold: 5
  skip_training: false
  epoch: 20
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: time_series
  group_name: time_id
  label_name: target
  feature_set:
  - f000
  dataset: ump_1
  model: ump_1
  pretrained: []
  criterion: RMSELoss
  optimizer: Adam
  scheduler: CosineAnnealingWarmupRestarts
  lr: 0.001
  min_lr: 1.0e-06
  weigh

In [9]:
import torch


In [10]:
import src.utils as utils

In [11]:
device = utils.gpu_settings(c)

2022-01-29 20:54:53,305 [INFO] [utils] CUDA_VISIBLE_DEVICES: 6,7
2022-01-29 20:54:53,406 [INFO] [utils] torch device: cuda, device count: 2


## Fold

In [12]:
from src.load_data import InputData

In [13]:
input = InputData(c)

2022-01-29 20:54:53,700 [INFO] [load_data] Load feather file. path: ../../inputs/train.f
2022-01-29 20:55:09,476 [INFO] [utils] Mem. usage decreased to 365Mb: 49% reduction
2022-01-29 20:55:09,513 [INFO] [load_data] Load feather file. path: ../../inputs/example_test.f
2022-01-29 20:55:09,671 [INFO] [utils] Mem. usage decreased to 0.0Mb: 49% reduction
2022-01-29 20:55:09,672 [INFO] [load_data] Load feather file. path: ../../inputs/example_sample_submission.f
2022-01-29 20:55:09,677 [INFO] [utils] Mem. usage decreased to 0.0Mb: 34% reduction


In [14]:
input.train

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,fold
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566,NaN
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624,NaN
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608,NaN
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309,NaN
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141405,1219_3768,1219,3768,0.033600,0.093530,-0.720275,-0.345497,-0.438781,-0.166972,-0.437182,...,-1.232434,-0.660579,0.875537,0.421628,-0.428097,-0.075548,-0.533092,-0.193732,-0.581394,4.0
3141406,1219_3769,1219,3769,-0.223264,-1.344935,-0.199987,-0.107702,-0.454677,-0.221914,-0.141174,...,-1.232434,-0.670493,0.875537,0.421628,-0.729949,-1.514277,0.013145,-0.890270,-0.589705,4.0
3141407,1219_3770,1219,3770,-0.559415,0.979489,-1.110491,1.006980,-0.467307,-0.159549,1.355671,...,-1.232434,0.820784,-1.142157,0.421628,-0.363329,1.363181,-0.079106,-1.580124,-0.297625,4.0
3141408,1219_3772,1219,3772,0.009599,-2.565332,0.320301,0.076600,1.380182,-0.155366,-0.689000,...,-1.232434,0.133074,-1.142157,0.421628,-0.375288,-1.514277,-0.973762,0.608647,-0.372040,4.0


## Base DNN

In [15]:
from src.make_model import make_model

In [16]:
model = make_model(c, device)

In [17]:
model

DataParallel(
  (module): BaseModel(
    (bn_1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fcs): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): Linear(in_features=300, out_features=300, bias=True)
      (2): Linear(in_features=300, out_features=300, bias=True)
      (3): Linear(in_features=300, out_features=300, bias=True)
    )
    (head): Linear(in_features=300, out_features=1, bias=True)
  )
)

In [18]:
input_ = torch.randn(2, 300).to(device)

In [19]:
model(input_)

ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/miniconda3/envs/py37-all-in-one/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/opt/miniconda3/envs/py37-all-in-one/lib/python3.7/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "../src/make_model.py", line 60, in forward
    x = self.bn_1(x)
  File "/opt/miniconda3/envs/py37-all-in-one/lib/python3.7/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/opt/miniconda3/envs/py37-all-in-one/lib/python3.7/site-packages/torch/nn/modules/batchnorm.py", line 140, in forward
    self.weight, self.bias, bn_training, exponential_average_factor, self.eps)
  File "/opt/miniconda3/envs/py37-all-in-one/lib/python3.7/site-packages/torch/nn/functional.py", line 2147, in ba